In [1]:
import os

In [2]:
%pwd

'/home/tejas/MLProj/Thyroid-Disease-Prediction/research'

In [3]:
os.chdir('../')

In [114]:
%pwd

'/home/tejas/MLProj/Thyroid-Disease-Prediction'

In [135]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/tejas05in/Thyroid-Disease-Prediction.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "tejas05in"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "9efcb5c7b79d0e949378459b922b1462a80fa413"

In [136]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [137]:
from ThyroidProject.constants import *
from ThyroidProject.utils.common import read_yaml , create_directories

In [138]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.GradientBoostedTreesLearner
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/tejas05in/Thyroid-Disease-Prediction.mlflow"
        )

        return model_evaluation_config

In [139]:
import os
import pandas as pd
from urllib.parse import urlparse
import mlflow
import numpy as np
import ydf
from ThyroidProject.utils.common import save_json

In [154]:
class CustomModelWrapper(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        import ydf
        model = ydf.load_model(
            context)
        return model

    def predict(self, context, model_input):
        return self.load_context(context).predict(model_input)


class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        
    

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = ydf.load_model(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_uri_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            
            evaluation = model.evaluate(test_data)

            # Saving metrics as local
            scores = {"accuray": evaluation.accuracy, "loss": evaluation.loss}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("loss",evaluation.loss)
            mlflow.log_metric("accuray",evaluation.accuracy)

            # Model registry does not work with file store
            if tracking_uri_type_store != 'file':

                # Register the model
                # There are other ways to use the model registry , which depends on the use case
                # Please refer to the documentation for more information
                # https://mlflow.org/docs/latest/model-registry.html #api-workflow
                mlflow.pyfunc.log_model("model",
                    python_model=CustomModelWrapper(),  # Use the custom model wrapper
                    # Log the model as an artifact
                    artifacts={"model": self.config.model_path},
                    conda_env="conda.yaml"
                )

            else:
                mlflow.pyfunc.log_model('model', python_model=CustomModelWrapper(), artifacts={"model": self.config.model_path})

In [155]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2024-01-27 16:38:47,421:INFO:common:yaml file: config/config.yaml loaded successfully]
[2024-01-27 16:38:47,423:INFO:common:yaml file: params.yaml loaded successfully]
[2024-01-27 16:38:47,425:INFO:common:yaml file: schema.yaml loaded successfully]
[2024-01-27 16:38:47,426:INFO:common:created directory at :artifacts]
[2024-01-27 16:38:47,426:INFO:common:created directory at :artifacts/model_evaluation]
[2024-01-27 16:38:47,432:INFO:model_lib:Loading model from artifacts/model_trainer/model]
[2024-01-27 16:38:47,971:INFO:common:json file saved at:{path}]
